In [89]:
### Ini adlah boolean
import math
class BooleanModel():
    @staticmethod
    def and_operation(left_operand, right_operand):
        # perfom 'merge'
        result = [] # reslts list to be returned
        l_index = 0 # current index in left_operand
        r_index = 0 # current index in right_operand
        l_skip =  int(math.sqrt(len(left_operand)))
        # skip pointer distance for l_index
        r_skip = int(math.sqrt(len(right_operand)))
        # skip pointer distance for r_index
        
        while(l_index < len(left_operand) and r_index <len(right_operand)):
            l_item = left_operand[l_index]
            r_item = right_operand[r_index]
            
            # case 1 : if match 
            if(l_item == r_item):
                result.append(l_item) # add to result 
                l_index += 1       # advance left index
                r_index += 1       # advance left index
                
            # case 2 : if left item is more than right item 
            elif (l_item > r_item):
                # if r_index can be skipped (if new r _index is still within range and resulting item is <= left item)
                if (r_index + r_skip <= len(right_operand)) and right_operand[r_index + r_skip] <= l_item :
                    r_index += r_skip
                # else advance r_index by 1
                else:
                    r_index += 1
                    
            # case 3 : if left item is less than right item 
            else: 
                # if l_index can be skipped (if new l_index is still within range and resulting item is < = right item)
                if(l_index + l_skip < len(left_operand)) and left_operand[l_index + l_skip] <= r_item:
                    l_index += l_skip
                #else advance l_index by 1
                else:
                    l_index +=1
        return(result)
    
    @staticmethod
    def or_operation(left_operand, right_operand):
        result = [] # union of left and right operand
        l_index = 0 # current index in left operand
        r_index = 0 # current index in right operand
        
        # while lists have not ten been covered
        while(l_index < len(left_operand) or r_index < len(right_operand)):
            # if both list are not yer exhausted
            if(l_index < len(left_operand) and r_index < len(right_operand)): 
                l_item = left_operand[l_index] # current item in left_operand
                r_item = right_operand[r_index] # current item in right_operand
                
                # case 1 : if items are equal, add either one to result and advance both pointers
                if(l_item == r_item):
                    result.append(l_item)
                    l_index += 1
                    r_index += 1
                
                # case 2: l_item greater than r_item, add r_item and advance r_index
                elif(l_item > r_item):
                    result.append(r_item)
                    r_index += 1
                
                # case 3 : l_item lower that r_item, add l_item and advance l_item 
                else:
                    result.append(l_item)
                    l_index += 1
                        
            # if left operand list is exhausted, append r_item and advance r_index
            elif(l_index >= len(left_operand)):
                r_item = right_operand[r_index]
                result.append(r_item)
                r_index += 1
                
            # else id right_operand list is exhausted, append l_item and advance l_index
            else:
                l_item = left_operand[l_index]
                result.append(l_item)
                l_index += 1
        return(result)
    
    @staticmethod
    def not_operation(right_operand, indexed_docIDs):
        # complement of an empty list is list of all indexedd docID
        if (not right_operand):
            return indexed_docIDs
                
                
        result = []
        r_index = 0 # index for right operand
        for item in indexed_docIDs:
            # if item do not match that in right_operand, it belongs to compliment 
            if(item != right_operand[r_index]):
                result.append(item)
            # else if item matches and r_index still can progress, advance it by 1
            elif(r_index + 1 < len(right_operand)):
                r_index += 1
        return(result)

In [91]:
def parse_query(infix_tokens):
    """ Parse Query
    Parsing done using Shunting Yard Algorithm
    """
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0
    
    output = []
    operator_stack = []
    
    for token in infix_tokens:
        if(token == '('):
            operator_stack.append(token)
        
        # if right bracket, pop all operators from operator stack onto output untill we hit left bracket 
        elif(token == ')'):
            operator = operator_stack.pop()
            while operator != '(':
                output.append(operator)
                operator = operator_stack.pop()
          
         # if operator, pop operators form operator stack to queue if they are of higher precedence
        elif(token in precedence):
            # if operator stack is not empty
            if (operator_stack):
                current_operator = operator_stack[-1]
                while(operator_stack and precedence[current_operator] > precedence[token]):
                    output.append(operator_stack.pop())
                    if(operator_stack):
                        current_operator = operator_stack[-1]
            operator_stack.append(token) # add token to stack 
        else:
            output.append(token.lower())
                
        # while there are still operators on the stack, pop them into the queue
    while(operator_stack):
        output.append(operator_stack.pop())     
    return (output) 
    
       

In [4]:
parse_query(["vaksin", "aku"])

['vaksin', 'aku']

In [5]:
# Code buat inverted Index

def stemming(text):
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    
    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    output = stemmer.stem(text)
    return(output)
    inverted_index = {}
def inverted_index(corpus_term):
    inverted_index = {}
    
    for i in range(len(corpus_term)):
        for item in corpus_term[i]:
            item = stemming(item)
            if item not in inverted_index: # inputing item in dictionary if not available
                inverted_index[item] = []
            if (item in inverted_index) and ((i+1) not in inverted_index[item]): # setelah ada di key dictionary, masukin indeksnya ke berapa aja
                inverted_index[item].append(i+1)
    return(inverted_index)


In [95]:
# import collections
# from boolean import BooleanModel


def process_query(query, n_docs, inverted_index):
    #prepare query list
    query = query.replace('(','( ')
    query = query.replace(')',' )')
    query = query.split(' ')
    print(query)

    indexed_docIDs = list(range(1, n_docs+1))
    results_stack = []
    postfix_queue = collections.deque(parse_query(query)) # get query in postfix notation as a queue
    print(postfix_queue)
    while postfix_queue:
        token = postfix_queue.popleft()
        print(token) #print the token of the query that we want to searching for
        result = [] # the evaluated result at each stage
        #if operand, add posting list for term to results stack 
        if (token != 'AND' and token != 'OR' and token != 'NOT'):
            token = stemming(token) #stem the token
            #default empty list if not in dictionary
            if(token in inverted_index):
                result = inverted_index[token]
                
        
        elif (token == 'AND'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = BooleanModel.and_operation(left_operand,right_operand) # evaluate and
        elif (token == 'OR'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = BooleanModel.or_operation(left_operand,right_operand)
        elif (token == 'NOT'):
            right_operand = result_stack.pop()
            result = BooleanModel.not_operation(right_operand, indexed_docIDs) #evaluate NOT
        print(result) # print the result of the Boolean Retrieval Model search the query
        results_stack.append(result)
        
    # NOTE : at this point results_stack should only have one item and it is the final result 
    if len(results_stack) != 1:
        print("ERROR : Invalid Query. Please check query syntax.")
        return None
    return results_stack.pop()


In [7]:
inv = {'virus': [1, 2, 3], 'sistem': [1, 3], 'informasi': [1], 'jadwal': [1], 'model': [2], 'analisis': [2, 3], 'sentimen': [2], 'berita': [2], 'input': [3], 'output': [3]}

In [94]:
print(f"Result the search of query is on document: {process_query('(informasi OR (analisis))', 3, inv)}")

['(', 'informasi', 'OR', '(', 'analisis', ')', ')']
deque(['informasi', 'analisis', 'OR'])
informasi
[1]
analisis
[2, 3]
OR
[1, 2, 3]
Result the search of query is on document: [1, 2, 3]


In [92]:
postfix_queue = collections.deque(parse_query(['analisis', 'OR', 'berita']))
print(postfix_queue)


deque(['analisis', 'berita', 'OR'])


In [88]:
indexed_docIDs = list(range(1, 3+1))
print(indexed_docIDs)

[1, 2, 3]


In [26]:
dump = ['1','2','3']
dump1 = dump.pop()
print(dump1)
print(dump)

3
['1', '2']


In [93]:
resultt_stack = []
resultt_stack.append("stack")
kanan_operand = resultt_stack.pop()
kiri_operand = resultt_stack.pop()

IndexError: pop from empty list

In [42]:
word= "word"
o  = word[2]
j = word [3]
print(type(o))
print(j)
if(o>j):
    print("a")

<class 'str'>
d
a
